In [ ]:
from PyPDF2 import PdfReader
import numpy as np
import pandas as pd
import nltk
import string
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
!pip install gensim
from gensim.parsing.preprocessing import remove_stopwords
!pip install tabulate
from tabulate import tabulate
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import pipeline
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\alice\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
# what we need to start 
company_list = ['10-KFord.pdf', '10-KTesla.pdf', '10-KStandard_Motor_Products.pdf', '10-KRivian.pdf', '10-KMotorcar_Parts_America.pdf', \
        '10-KMS.pdf', '10-KSilicon.pdf', '10-KJPMorgan.pdf', '10-KCityGroup.pdf', '10-KBankofAmerica.pdf',\
        '10-KChevron.pdf', '10-KConoco.pdf', '10-KGeneral_Electric.pdf', '10-KPetrogas.pdf', '10-KRoyal_Energy.pdf',\
        '10-KBeyond_Meat.pdf', '10-KCampbell.pdf', '10-KHershey.pdf', '10-KKellogg.pdf', '10-KCMcDonalds.pdf']

# file_list = ['file_1.txt', 'file_2.txt', 'file_3.txt', 'file_4.txt', 'file_5.txt', 'file_6.txt',\
#              'file_7.txt', 'file_8.txt', 'file_9.txt', 'file_10.txt', 'file_11.txt', 'file_12.txt',\
#                  'file_13.txt', 'file_14.txt', 'file_15.txt', 'file_16.txt', 'file_17.txt', \
#                      'file_18.txt', 'file_19.txt', 'file_20.txt' ]

social = pd.read_excel('social.xlsx')
environment = pd.read_excel('environment.xlsx')
economy = pd.read_excel('economy.xlsx')
social_words = social['social words'].tolist()
environment_words = environment['environment words'].tolist()
economy_words = economy['economy words'].tolist()
words_list = social_words + environment_words + economy_words

stop_words = set(stopwords.words("english"))

cleaner = re.compile('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')

# LM_dictionary = pd.read_excel('LM_MasterDictionary.xlsx', dtype={'Word': str})
# neg = LM_dictionary[LM_dictionary['Negative']>0]['Word'].tolist()
# pos = LM_dictionary[LM_dictionary['Positive']>0]['Word'].tolist()

In [ ]:
# functions
def extract(pdf_file:str):
    file_read = PdfReader(pdf_file)
    pdf_text=""
    for page in file_read.pages:
        content = page.extract_text()
        pdf_text += content
    return pdf_text 

def cleanhtml(raw_html):
  cleantext = re.sub(cleaner, '', raw_html)
  return cleantext

def remove_numbers(text):
    result = re.sub(r'\d+', '', text)
    return result

def remove_punctuation(text):
    translator = str.maketrans('', '', string.punctuation)
    return text.translate(translator)

def remove_whitespace(text):
    return  " ".join(text.split())

def esg_percentage(text_file,list_of_words):
    count = 0
    text_split = text_file.split()
    n = len(text_split)
    for i in range(0,n):
        for word in list_of_words:
            if text_split[i]==word.lower(): 
                count += 1 
            elif i<n-1 and text_split[i]+" "+text_split[i+1]==word.lower(): 
                count += 1 
            elif i<n-2 and text_split[i]+" "+text_split[i+1]+" "+text_split[i+2]==word.lower():
                count += 1 
            elif i<n-3 and text_split[i]+" "+text_split[i+1]+" "+text_split[i+2]+" "+text_split[i+3]==word.lower():
                count += 1
    percentage_words = count/n
    return percentage_words

def clean_list(list,list_of_words):
    new_list = []
    accepted = 0 
    total_number_phrases = len(list)
    for phrase in list:
        phrase = remove_punctuation(phrase) # remove punctuation
        phrase = re.sub(r'[^\w]', ' ', phrase) # remove all character that are different from words
        words_list = word_tokenize(phrase) # create a list of words from the string phrase
        filtered_words_list = [word for word in words_list if word not in stop_words] # remove stopwords
        count = 0
        n = len(filtered_words_list)
        for i in range(0,n):
            for word in list_of_words:
                if filtered_words_list[i]==word.lower(): 
                    count += 1 
                elif i<n-1 and filtered_words_list[i]+" "+filtered_words_list[i+1]==word.lower(): 
                    count += 1 
                elif i<n-2 and filtered_words_list[i]+" "+filtered_words_list[i+1]+" "+filtered_words_list[i+2]==word.lower():
                    count += 1 
                elif i<n-3 and filtered_words_list[i]+" "+filtered_words_list[i+1]+" "+filtered_words_list[i+2]+" "+filtered_words_list[i+3]==word.lower():
                    count += 1
        if count != 0: 
            accepted += 1
            phrase = ' '.join(filtered_words_list) # create again the string 
            new_list = new_list + [phrase]
    return new_list, accepted/total_number_phrases

def count_neg_pos(text):
    count_neg = 0
    count_pos = 0 
    for word in word_tokenize(text):
        count_neg += 1 if word.upper() in neg else 0
        count_pos += 1 if word.upper() in pos else 0
    return (count_pos-count_neg)/(count_pos+count_neg)

def bert_phrase(list):
    finbert = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-tone',num_labels=3)
    tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-tone')
    nlp = pipeline("sentiment-analysis", model=finbert, tokenizer=tokenizer)

    n = len(list)
    count_pos = 0
    count_neg = 0 
    count_neut = 0 
    score_pos = 0
    score_neg = 0
    score_neut = 0
    compound = 0 
    if n>0: 
        for sentence in list:
            if len(sentence)<512:
                result = nlp(sentence)
                label_result=result[0]['label']
                count_pos = ((label_result=='Positive')+count_pos)
                count_neg =  ((label_result=='Negative')+count_neg)
                count_neut =  ((label_result=='Neutral')+count_neut)
            else:
                n=n-1
        compound=(count_pos*1 + count_neg *(-1))/n
        count_pos=count_pos/n
        count_neg=count_neg/n
        count_neut=count_neut/n
    array = np.array([compound, count_pos, count_neg, count_neut, n/len(list)])
    return array 
#Bert doesn't work for sentences of more than 512 words; I just drop them
#the percentage of such 'too long' sentences is the last value of the returned array

In [ ]:
list_companies = ['Ford', 'Tesla', 'Standard_Motor_Products', 'Rivian', 'Motorcar_Parts_America', 'MS', 'Silicon', 'JPMorgan', 'CityGroup', 'BankofAmerica',\
        'Chevron', 'Conoco', 'General_Electric', 'Petrogas', 'Royal_Energy','Beyond_Meat', 'Campbell', 'Hershey', 'Kellogg', 'McDonalds']
df_phrases = pd.DataFrame(columns=['average_compound','perc_pos','perc_neg','perc_neut', 'perc_esg_phrases'], index=list_companies)
df_document = pd.DataFrame(columns=['sentiment'], index=list_companies)
count = 0
for i in company_list: 
    x = extract(i)
    x = cleanhtml(x)
    x = re.sub(r'''(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))''', " ", x)
    x= x.lower()
    x= remove_numbers(x)
    x = x.split(".")
    x,acceptance_rate = clean_list(x,words_list)
    bert_phrases = bert_phrase(x)
    df_phrases.loc[list_companies[count]] = pd.Series({'average_compound': bert_phrases[0],'perc_pos': bert_phrases[1], 'perc_neg': bert_phrases[2], 'perc_neut': bert_phrases[3], 'perc_esg_phrases': acceptance_rate})
    count += 1

In [ ]:
df_phrases

,average_compound,perc_pos,perc_neg,perc_neut,perc_esg_phrases
Ford,-0.07393,0.107004,0.180934,0.712062,0.110452
Tesla,-0.005988,0.164671,0.170659,0.664671,0.118833
Standard_Motor_Products,-0.024814,0.133995,0.158809,0.707196,0.158033
Rivian,-0.131737,0.155689,0.287425,0.556886,0.144356
Motorcar_Parts_America,-0.035608,0.124629,0.160237,0.715134,0.163343
MS,-0.095327,0.037383,0.13271,0.829907,0.112245
Silicon,-0.047157,0.047157,0.094313,0.85853,0.125061
JPMorgan,-0.107865,0.035955,0.14382,0.820225,0.094653
CityGroup,-0.039683,0.056689,0.096372,0.846939,0.089598
BankofAmerica,0.018868,0.018868,0.0,0.981132,0.129032


In [ ]:
df_phrases.to_csv('Sentimental_Bert_on_10K.csv')